# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
from transformers import AutoTokenizer #, AutoModelForSequenceClassification

In [6]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [7]:
from util import util_pipeline

In [8]:
import importlib
# importlib.reload(util_elastic)

In [9]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [10]:
PATH_DOC =  '../data/juris_tcu_index/doc.csv'
PATH_QUERY =  '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

In [11]:
%%time
import logging
logging.getLogger("haystack").setLevel(logging.WARNING) #WARNING, INFO

CPU times: user 186 µs, sys: 15 µs, total: 201 µs
Wall time: 203 µs


# Data load

In [12]:
df_doc = pd.read_csv(PATH_DOC)

In [13]:
df_query = pd.read_csv(PATH_QUERY)

In [14]:
df_qrel = pd.read_csv(PATH_QREL)

In [15]:
df_doc.shape, df_query.shape, df_qrel.shape

((13255, 28), (16045, 11), (94809, 3))

In [16]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


### Quantitative calculation

Tokenization loading

In [17]:
nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_pt = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
assert os.path.exists(nome_caminho_modelo_pt), f"Path para {nome_caminho_modelo_pt} não existe!"

nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")

nome_caminho_modelo_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_3b), f"Path para {nome_caminho_modelo_3b} não existe!"

tokenizador_pt_minilm = AutoTokenizer.from_pretrained(nome_caminho_modelo_pt)
tokenizador_pt_monot5_3b = AutoTokenizer.from_pretrained(nome_caminho_modelo_3b)

/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [18]:
nome_modelo_ptt5_base = 'unicamp-dl/ptt5-base-pt-msmarco-100k-v2'
nome_caminho_modelo_ptt5_base  = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ptt5_base
assert os.path.exists(nome_caminho_modelo_ptt5_base ), f"Path para {nome_caminho_modelo_ptt5_base } não existe!"

In [19]:
tokenizador_ptt5_base = AutoTokenizer.from_pretrained(nome_caminho_modelo_ptt5_base)

In [20]:
def retorna_num_tokens(parm_texto:str, parm_tokenizador:AutoTokenizer):
    return len(parm_tokenizador.tokenize(parm_texto))


In [21]:
df_query['LEN_TEXT_CHAR'] = df_query['TEXT'].apply(len)
df_query['LEN_TEXT_CHAR_LOG'] = round(np.log(df_query['TEXT'].apply(len))).astype(int)
df_query['NUM_WORD'] = df_query['TEXT'].apply(lambda x: len(x.split()))
df_query['NUM_TOKENS_MONOT5_3B'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_monot5_3b)
df_query['NUM_TOKENS_MINILM'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_minilm)
df_query['NUM_TOKENS_PTT5_BASE'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_ptt5_base)


In [22]:
df_query.columns, df_qrel.columns, df_doc.columns, 

(Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
        'NORMATIVE_AUTHOR_NAME', 'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM', 'NUM_TOKENS_PTT5_BASE'],
       dtype='object'),
 Index(['QUERY_ID', 'DOC_ID', 'TYPE'], dtype='object'),
 Index(['TEXT', 'ID', 'NAME', 'DATE_REFERENCE', 'CLASS', 'HAS_SCOPE_NOTE', 'HAS_DEFINITION', 'HAS_SYNONYM', 'HAS_EXAMPLE', 'HAS_ENGLISH_TRANSLATION', 'HAS_SPECIALIZATION', 'HAS_RELATED_TERM',
        'HAS_SPANISH_TRANSLATION', 'HAS_GENERALIZATION', 'TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION',
        'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION', 'COUNT_INDEX_AREA', 'COUNT_INDEX_THEME', 'COUNT_INDEX_SUBTHEME', 'COUNT_INDEX_EXTRA', 'COUNT_INDEX_TOTAL'],
       dtype='object'

In [23]:
# Verificar as colunas com valores NaN
columns_with_nan = df_query.columns[df_query.isna().any()].tolist()

print(columns_with_nan)

['REFERENCE_LIST', 'PARADIGMATIC', 'NORMATIVE_PROCESS_TYPE']


In [24]:
df_query = df_query.fillna("")

In [25]:
df_doc.head()

,TEXT,ID,NAME,DATE_REFERENCE,CLASS,HAS_SCOPE_NOTE,HAS_DEFINITION,HAS_SYNONYM,HAS_EXAMPLE,HAS_ENGLISH_TRANSLATION,HAS_SPECIALIZATION,HAS_RELATED_TERM,HAS_SPANISH_TRANSLATION,HAS_GENERALIZATION,TEXT_DEFINITION,TEXT_SYNONYM,TEXT_RELATED_TERM,TEXT_SCOPE_NOTE,TEXT_EXAMPLE,TEXT_ENGLISH_TRANSLATION,TEXT_SPANISH_TRANSLATION,TEXT_SPECIALIZATION,TEXT_GENERALIZATION,COUNT_INDEX_AREA,COUNT_INDEX_THEME,COUNT_INDEX_SUBTHEME,COUNT_INDEX_EXTRA,COUNT_INDEX_TOTAL
0,"O termo é ""Abandono de cargo"".\nAbandono de cargo tem definição: ""Configura abandono de cargo a ...",1,Abandono de cargo,20230614,Termo,1,1,1,0,1,0,1,1,0,Configura abandono de cargo a ausência intencional do servidor ao serviço por mais de trinta dia...,Abandono de função e Abandono do cargo.,"Abandono de emprego, Deserção, Falta grave, Função pública, Demissão de servidor, Inassiduidade ...",Artigo 323 Código Penal,NaN,Abandonment of office.,Abandono de puesto.,NaN,NaN,0,1,0,0,1
1,"O termo é ""Abastecimento de tropas e meios"".\nAbastecimento de tropas e meios é uma especializaç...",2,Abastecimento de tropas e meios,20230614,Termo,0,0,0,0,1,0,0,1,1,NaN,NaN,NaN,NaN,NaN,Supply of troops and means.,Abastecimiento de tropas y medios e Suministro de tropas y médios.,Abastecimento.,NaN,0,0,0,1,1
2,"O termo é ""Tropa"".\nTropa tem definição: ""Termo coletivo que designa o pessoal de uma organizaçã...",3,Tropa,20230614,Termo,0,1,0,0,1,0,1,1,0,Termo coletivo que designa o pessoal de uma organização militar.,NaN,Organização militar e Missão.,NaN,NaN,Troop.,Tropa.,NaN,NaN,0,0,0,0,0
3,"O termo é ""Abono de faltas"".\nAbono de faltas tem definição: ""É a relevação da falta, de forma q...",4,Abono de faltas,20230614,Termo,1,1,1,0,0,0,1,0,0,"É a relevação da falta, de forma que o servidor não sofra qualquer desconto pelo dia não trabalh...","Faltas, Abono das faltas e Abono de ausências.",Inassiduidade habitual e Afastamento de pessoal.,Usado para servidores regidos pela CLT (art. 473).,NaN,NaN,NaN,NaN,NaN,0,0,0,1,1
4,"O termo é ""Abono de permanência em serviço"".\nAbono de permanência em serviço tem definição: ""O ...",5,Abono de permanência em serviço,20230614,Termo,0,1,1,0,0,0,1,0,0,O abono de permanência será pago ao servidor que tiver preenchido as exigências para aposentador...,"Abono de permanência, Adicional de permanência e Abono permanência.","Aposentadoria voluntária, Contribuição previdenciária e Regime Próprio de Previdência Social.",NaN,NaN,NaN,NaN,NaN,NaN,0,7,0,4,11


In [26]:
# Verificar as colunas com valores NaN
columns_with_nan = df_doc.columns[df_doc.isna().any()].tolist()

print(columns_with_nan)

['TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION', 'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION']


In [27]:
df_doc = df_doc.fillna("")

In [28]:
# Converter as colunas para o tipo string
df_doc[columns_with_nan] = df_doc[columns_with_nan].astype(str)

# Expand docs

## Create reference to index (juris_tcu_index)

In [29]:
INDEX_NAME = 'indir_juris_tcu_index'

In [30]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13255

Qtd de embeddings 13255

Documento.id=5: <Document: id=5, content='O termo é "Abono de permanência em serviço".
Abono de permanência em serviço tem definição: "O abono...'>


## Create pipeline

In [31]:
# Test query
# parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou."

In [32]:
pipes = []

In [33]:
ranker_type = 'PTT5_INDIR_400' #'PTT5_BASE' #'PTT5_INDIR_83' # 'MT5_3B' # 'MT5_TRAINED_LIM50_800' # 'MINILM_TRAINED_19000'4

In [34]:
FILTER_TERMO = {'class':['Termo']}

In [35]:
FILTER_TERMO, ranker_type

({'class': ['Termo']}, 'PTT5_INDIR_400')

In [36]:
%%time
pipe_join_bm25_sts_reranker = util_pipeline.return_pipeline_join_bm25_sts_reranker(index, ranker_type, parm_limit_query_size=350)

Loading PTT5_INDIR_400 with limit_query_size=350
CPU times: user 11.6 s, sys: 1.82 s, total: 13.4 s
Wall time: 7.14 s


In [37]:
pipes.append({'PIPE_NAME': 'pipe_join_bm25_sts_reranker',
              'PIPE_OBJECT': pipe_join_bm25_sts_reranker,
              'RETRIEVER_TYPE': 'join_sts_bm25',  
              'RETRIEVER_MODEL_NAME': '', 
              'RANKER_MODEL_NAME': util_pipeline.return_ranker_name(ranker_type)})

In [42]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 150}, "StsRetriever": {"top_k": 150}, "Ranker": {"top_k": 20}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 20)

Parâmetros usados: {'Bm25Retriever': {'top_k': 150}, 'StsRetriever': {'top_k': 150}, 'Ranker': {'top_k': 20}, 'filters': {'class': ['Termo']}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 20
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Transferência documental', '6139', -0.005041031166911125]
1 : ['Armazenamento de dados', '1101009', -0.01332791242748499]
2 : ['Vazamento de dados', '1110487', -0.017766617238521576]
3 : ['Determinação', '15266', -0.02106134407222271]
4 : ['Gestor', '716', -0.061831917613744736]
5 : ['Gestão documental', '6136', -0.0694652572274208]
6 : ['Culpa in vigilando', '360', -0.23385845124721527]
7 : ['Competência do TCU', '5095', -0.25020989775657654]
8 : ['Gestor público', '5307', -0.2594102621078491]
9 : ['Conduta omis

In [43]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 100}, "StsRetriever": {"top_k": 100}, "Ranker": {"top_k": 20}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 20)

Parâmetros usados: {'Bm25Retriever': {'top_k': 100}, 'StsRetriever': {'top_k': 100}, 'Ranker': {'top_k': 20}, 'filters': {'class': ['Termo']}}
Consulta: A transferência de documentos da entidade para local impróprio ao armazenamento, causando a perda de informações ou inviabilizando seu manuseio, de forma a impedir a atuação do TCU, é causa de responsabilização do gestor que a ordenou.
Qtd documentos retornados: 20
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Armazenamento de dados', '1101009', -0.01332791242748499]
1 : ['Vazamento de dados', '1110487', -0.017766617238521576]
2 : ['Determinação', '15266', -0.02104850485920906]
3 : ['Gestor', '716', -0.061831917613744736]
4 : ['Gestão documental', '6136', -0.069437675178051]
5 : ['Culpa in vigilando', '360', -0.23364831507205963]
6 : ['Competência do TCU', '5095', -0.25020989775657654]
7 : ['Conduta omissiva', '15275', -0.2904728055000305]
8 : ['Arquivamento', '108', -0.30233100056648254]
9 : ['Controle interno', '339'

In [40]:
raise Exception('Stop here!')

Exception: Stop here!

## Create expansion

In [44]:
from unidecode import unidecode
import string
import re


In [45]:
def set_normalize_text(text):
    # Remover espaços duplicados e espaços extras à direita e à esquerda
    text = re.sub(r'\s+', ' ', text.lower()).strip()
    # Remover acentos e caracteres especiais
    text = unidecode(text)
    # Separar as palavras em substrings
    words = text.split()
    # Retornar um conjunto de palavras normalizadas
    return set(words)

In [46]:
def test_similar_or_subtext(text1, text2):
    normalized_text1 = set_normalize_text(text1)
    normalized_text2 = set_normalize_text(text2)
    return normalized_text1 == normalized_text2 or normalized_text1.issubset(normalized_text2)


In [47]:
set_normalize_text('olá comO   vai voc? está bem.')

{'bem.', 'como', 'esta', 'ola', 'vai', 'voc?'}

In [48]:
test_similar_or_subtext("Licitação de técnica e preço","técnica e preço")

False

In [107]:

def expand_query(parm_pipe, parm_df_query, num_expansion):
    if num_expansion > 30:
        raise Exception('Review limit in code for num_expansion of 30!')
    count_expansion = [inteiro + 1 for inteiro in range(num_expansion)]
    expanded_rows = []
    for _, row in tqdm(parm_df_query.iterrows(), total =parm_df_query.shape[0]):
        query_id = row['ID']
        doc_return = parm_pipe.run(query=row['TEXT'], params={"Bm25Retriever": {"top_k": 150}, "StsRetriever": {"top_k": 150}, "filters": FILTER_TERMO}) # , "Ranker": {"top_k": num_expansion*4}

        termos = []
        text_formed = row['TEXT']
        for docto in doc_return['documents'][:num_expansion * 3]:
            if not test_similar_or_subtext(docto.meta['name'], text_formed):
                termos.append(docto.meta['id'])
            if len(termos) == num_expansion:
                break
        
        for count in count_expansion:
            if count <= len(termos):
                doc_id = termos[count-1]
                expanded_rows.append({'QUERY_ID': query_id, 'DOC_ID': doc_id, 'ORDER': count})
    
    df_expanded = pd.DataFrame(expanded_rows)
    return df_expanded


In [108]:
df_query = pd.read_csv(PATH_QUERY)

In [109]:
df_query.head(2)

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE


%%time
df_expanded = expand_query(parm_pipe = pipe_join_bm25_sts_reranker, parm_df_query=df_query.sample(frac=0.0001, random_state=123).reset_index(drop=True), num_expansion=5)

In [112]:
%%time
df_expanded = expand_query(parm_pipe = pipe_join_bm25_sts_reranker, parm_df_query=df_query, num_expansion=5)

100%|██████████| 16045/16045 [10:43:51<00:00,  2.41s/it] 

CPU times: user 10h 9min 20s, sys: 7.51 s, total: 10h 9min 28s
Wall time: 10h 43min 51s


In [113]:
EXPANSOR_CRITERIA = 'join_300_ptt5_indir_400' 

In [114]:
PATH_DOC_EXPANDED =  f'../data/juris_tcu_expanded_doc/doc_5_expansion_{EXPANSOR_CRITERIA}.csv' 

In [115]:
df_expanded.to_csv(PATH_DOC_EXPANDED, index=False)

In [116]:
df_expanded.head()

,QUERY_ID,DOC_ID,ORDER
0,13568,94,1
1,13568,14803,2
2,13568,3302,3
3,13568,96,4
4,13568,2021,5


In [117]:
df_expanded.shape

(80221, 3)

# Generate concatenations

Concatenate terms (by qrel) and synonyms (by doc)

In [118]:
df = df_expanded.merge(df_doc[['ID', 'NAME', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM']], how='inner', left_on='DOC_ID', right_on='ID')

In [119]:
df.shape

(80221, 9)

In [120]:
df.head(2)

,QUERY_ID,DOC_ID,ORDER,ID,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM
0,13568,94,1,94,Aposentadoria especial,,"Regime Geral de Previdência Social, Salário-de-benefício e Perfil profissiográfico previdenciário.",0,1
1,8529,94,2,94,Aposentadoria especial,,"Regime Geral de Previdência Social, Salário-de-benefício e Perfil profissiográfico previdenciário.",0,1


In [121]:
del df['ID']

In [145]:
df[df['TEXT_SYNONYM']==''].shape

(30502, 22)

In [185]:
df['NAME_SYNONYM'] = df.apply(lambda row: row['NAME'] if row['TEXT_SYNONYM'] == '' else row['NAME'] + ' - ' + row['TEXT_SYNONYM'].rstrip('.'), axis=1)


In [186]:
df['NAME_RELATED_TERM'] = df.apply(lambda row: row['NAME'] if row['TEXT_RELATED_TERM'] == '' else row['NAME'] + ' -- ' + row['TEXT_RELATED_TERM'].rstrip('.'), axis=1).str.rstrip('.')


In [187]:
df['NAME_SYNONYM_RELATED_TERM'] = df.apply(lambda row: row['NAME_SYNONYM'] if row['TEXT_RELATED_TERM'] == '' else row['NAME_SYNONYM'].rstrip('.') + ' -- ' + row['TEXT_RELATED_TERM'].rstrip('.'), axis=1).str.rstrip('.')


In [188]:
df[(df['HAS_SYNONYM']==1) & (df['HAS_RELATED_TERM']==0)].head(1)

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,QUERY_ID,DOC_ID,ORDER,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM,NAME_SYNONYM,NAME_RELATED_TERM,NAME_SYNONYM_RELATED_TERM
15,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",,,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ,35016,1101432,1,Alta administração,"Alta direção, Tomadores de decisão e Decisores.",,1,0,"Alta administração - Alta direção, Tomadores de decisão e Decisores",Alta administração,"Alta administração - Alta direção, Tomadores de decisão e Decisores"


In [129]:
df = df_query.merge(df, how='inner', left_on='ID', right_on='QUERY_ID')

In [130]:
df.columns

Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
       'NORMATIVE_AUTHOR_NAME', 'QUERY_ID', 'DOC_ID', 'ORDER', 'NAME', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM', 'NAME_RELATED_TERM',
       'NAME_SYNONYM_RELATED_TERM'],
      dtype='object')

In [85]:
df.shape

(10, 21)

In [133]:
# Verificar as colunas com valores NaN
columns_with_nan = df.columns[df.isna().any()].tolist()

print(columns_with_nan)

['REFERENCE_LIST', 'PARADIGMATIC', 'NORMATIVE_PROCESS_TYPE']


In [134]:
df = df.fillna("")

In [135]:
# Converter as colunas para o tipo string
df[columns_with_nan] = df[columns_with_nan].astype(str)

In [136]:
df.head(1)

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,QUERY_ID,DOC_ID,ORDER,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM,NAME_SYNONYM,NAME_RELATED_TERM,NAME_SYNONYM_RELATED_TERM
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE,13568,94,1,Aposentadoria especial,,"Regime Geral de Previdência Social, Salário-de-benefício e Perfil profissiográfico previdenciário.",0,1,Aposentadoria especial,"Aposentadoria especial -- Regime Geral de Previdência Social, Salário-de-benefício e Perfil prof...","Aposentadoria especial -- Regime Geral de Previdência Social, Salário-de-benefício e Perfil prof..."


## GERAR ATÉ 1, 3 e 5

Com as 5 expansões

In [194]:
%%time
data_index = {}
for cnt_expansion in [1, 3, 5]:
    data_index[cnt_expansion] = {'term' : [],
                'synonym': [],
                'related_term': [],
                'synonym_related_term': [],}
    # Ordenar o DataFrame por 'QUERY_ID' em ordem ascendente
    df_sorted = df[df['ORDER']<=cnt_expansion].sort_values('QUERY_ID')
    # Iterar sobre os grupos
    for cnt, (query_id, group) in tqdm(enumerate(df_sorted.groupby('QUERY_ID'))):
        # print(f"Query ID: {query_id}")
        # print(group)
        group_synonym = ''
        group_term = ''
        group_related_term = ''
        group_synonym_related_term = ''

        # Iterar sobre os registros do grupo
        for cnt, (_, row_doc) in enumerate(group.iterrows()):
            if cnt==0: # salva copia dos dados da query
                row_query = row_doc.copy()

            doc_id = row_doc['DOC_ID']
            # Faça o que precisa com cada registro (doc_id) para o query_id atual
            
            # Exemplo: Imprimir o doc_id
            # print(f"Doc ID: {doc_id}")

            group_term = group_term + ', ' + row_doc['NAME'] if group_term != '' else row_doc['NAME']
            #if row_doc['HAS_SYNONYM']==1:
            group_synonym = group_synonym + '; ' + row_doc['NAME_SYNONYM'] if group_synonym != '' else row_doc['NAME_SYNONYM']
            #if row_doc['HAS_RELATED_TERM']==1:
            group_related_term = group_related_term + '; ' + row_doc['NAME_RELATED_TERM'] if group_related_term != '' else row_doc['NAME_RELATED_TERM']
            # if (row_doc['HAS_RELATED_TERM']==1) or (row_doc['HAS_SYNONYM']==1):
            group_synonym_related_term = group_synonym_related_term + '; ' + row_doc['NAME_SYNONYM_RELATED_TERM'] if group_synonym_related_term != '' else row_doc['NAME_SYNONYM_RELATED_TERM']
        # Adicione aqui qualquer lógica adicional que você precise executar para cada query_id

        row_json_term= {
            'id': row_query['ID'],
            'content': row_query['TEXT'],
            'meta': {},
        }               
        for column_name in df.columns:
            if column_name not in ['TEXT', 'QUERY_ID', 'DOC_ID', 'TYPE', 'NAME', 'TEXT_SYNONYM',
                                'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM',
                                'NAME_RELATED_TERM', 'NAME_SYNONYM_RELATED_TERM']:  # columns of doc
                row_json_term['meta'][column_name.lower()] = row_query[column_name]        
        
        row_json_synonym = row_json_term.copy()
        row_json_related_term = row_json_term.copy()
        row_json_synonym_related_term = row_json_term.copy()

        row_json_term['content'] += '\n' + group_term # + '.'
        #if len(group_synonym) > 0:
        row_json_synonym['content'] += '\n' + group_synonym
        #if len(group_related_term) > 0:
        row_json_related_term['content'] += '\n' + group_related_term
        # if len(group_synonym_related_term) > 0:
        row_json_synonym_related_term['content'] += '\n' + group_synonym_related_term


        data_index[cnt_expansion]['term'].append(row_json_term)
        data_index[cnt_expansion]['synonym'].append(row_json_synonym)
        data_index[cnt_expansion]['related_term'].append(row_json_related_term)
        data_index[cnt_expansion]['synonym_related_term'].append(row_json_synonym_related_term)


16045it [00:03, 4910.69it/s]
16045it [00:04, 3471.24it/s]
16045it [00:06, 2643.67it/s]

CPU times: user 14 s, sys: 148 ms, total: 14.1 s
Wall time: 14 s


In [195]:
len(data_index[3]['term'])

16045

In [175]:
list_type_index = ['term', 'synonym', 'related_term', 'synonym_related_term' ]

Ajustado em 9/7: se não tiver sinônimo, trazer o termo mesmo assim

In [199]:
ind_test=30
for index_type in list_type_index:
    print(index_type, data_index[1][index_type][ind_test]['content'], '\n')


term SÚMULA TCU 31: É permitido ao aposentado rever, a qualquer tempo, a opção ensejada pelo artigo 180, § 2º, da Lei nº 1.711, de 28/10/52.
Aposentadoria-prêmio 

synonym SÚMULA TCU 31: É permitido ao aposentado rever, a qualquer tempo, a opção ensejada pelo artigo 180, § 2º, da Lei nº 1.711, de 28/10/52.
Aposentadoria-prêmio - Vantagem do artigo 192 da Lei 8.112/90, Vantagem do art. 192 da Lei 8.112/90, Vantagem do art. 184 da Lei 1.711/52, Aposentadoria prêmio e Vantagem do artigo 184 da Lei 1.711/52 

related_term SÚMULA TCU 31: É permitido ao aposentado rever, a qualquer tempo, a opção ensejada pelo artigo 180, § 2º, da Lei nº 1.711, de 28/10/52.
Aposentadoria-prêmio -- Reforma-prêmio 

synonym_related_term SÚMULA TCU 31: É permitido ao aposentado rever, a qualquer tempo, a opção ensejada pelo artigo 180, § 2º, da Lei nº 1.711, de 28/10/52.
Aposentadoria-prêmio - Vantagem do artigo 192 da Lei 8.112/90, Vantagem do art. 192 da Lei 8.112/90, Vantagem do art. 184 da Lei 1.711/52, Apo

# Index creation

In [154]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu', 'uuid': 'RD_aAWppQH6vjNxKUdL_0w', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '16045', 'store.size': '334.9mb', 'pri.store.size': '334.9mb'}

Index: indir_juris_tcu_term
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_term', 'uuid': 'S5WlNGiqSXC-79WaqfsddA', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '16045', 'store.size': '337.5mb', 'pri.store.size': '337.5mb'}

Index: indir_juris_tcu_synonym_related_term
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_synonym_related_term', 'uuid': 'OJIJ0P2XQo-s4NRuaT4W_w', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '3045', 'store.size': '348.4mb', 'pri.store.size': '348.4mb'}

Index: indir_juris_tcu_related_term
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_related_term', 'uuid': 'i9nzlBQrREyEC7MGT79GBQ', 'pri': '1', 'rep': '1', 'docs.count': '16045',

In [155]:
INDEX_NAME = 'indir_juris_tcu'

In [156]:
ranker_type

'PTT5_INDIR_400'

indir_juris_tcu_synonym_exp_5_ptt5_indir_400
indir_juris_tcu_exp_5_ptt5_indir_400_synonym

In [157]:
expanded_count = 5

In [ ]:
# util_elastic.delete_index('indir_juris_tcu')

In [203]:
%%time
dict_index = {}
for cnt_expansion in data_index:
    for index_type in list_type_index:
        index_name_with_expansion = INDEX_NAME+'_'+index_type +'_exp_'+str(cnt_expansion)+'_'+ranker_type.lower()
        print(f"\n********Creating index {index_name_with_expansion}")
        dict_index[index_name_with_expansion] = util_elastic.create_index(parm_index_name=index_name_with_expansion,
                                                           parm_data_carga_json=data_index[cnt_expansion][index_type],
                                                           parm_embedding_dim=1024)


********Creating index indir_juris_tcu_term_exp_1_ptt5_indir_400

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>

********Creating index indir_juris_tcu_synonym_exp_1_ptt5_indir_400

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>

********Creating index indir_juris_tcu_related_term_exp_1_ptt5_indir_400

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem 

In [206]:
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"


In [208]:
%%time
for index_name in dict_index:
    print(f"\n********Updating embeddings of index {index_name}")
    util_elastic.update_index_embedding_sts(parm_index=dict_index[index_name], parm_path_model=nome_caminho_modelo)


********Updating embeddings of index indir_juris_tcu_term_exp_1_ptt5_indir_400


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym_exp_1_ptt5_indir_400


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_related_term_exp_1_ptt5_indir_400


...


********Updating embeddings of index indir_juris_tcu_related_term_exp_5_ptt5_indir_400


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym_related_term_exp_5_ptt5_indir_400


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]

CPU times: user 22min 27s, sys: 12.2 s, total: 22min 39s
Wall time: 32min 29s
